In [ ]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
import glob 
from scipy.io.wavfile import read
import IPython.display as ipd
import librosa
import librosa.display
import librosa.feature
import pickle
from scipy import signal

In [ ]:
#Loading unnoised data without silence samples

X=np.load('intermediate_pickles/X.pkl')
Y=np.load('intermediate_pickles/Y.pkl')

print(X.shape)
print(Y.shape)

In [ ]:
#Loading noise 
X_n = pickle.load( open( "intermediate_pickles/X_noise_shuffled.pkl", "rb" ) )
Y_n = pickle.load( open( "intermediate_pickles/Y_noise_shuffled.pkl", "rb" ) )

In [ ]:
known=0
for lab in range(0,10):
    known+=len(np.where(Y==lab)[0])

X_time_shifted = np.zeros((known,X.shape[1]))
Y_time_shifted=[]

#The signals will be right shifted 50 percent and left shifted 50 percent of the times
prob_right_shift = 0.5

get = 0

for samp in range(0,X.shape[0]): 
    
    #Only for known data
    if Y[samp]!=10 and Y[samp]!=11:
        
        rand = 0.01*np.random.randint(100)

        #Right shifting 
        if rand > prob_right_shift: 

            #Random number between 1 to 200 ms
            shift = 0.001*np.random.randint(0,200)

            series = X[samp]
            series_shif = np.roll(series,int(shift*16000))
            X_time_shifted[get]= series_shif
            Y_time_shifted.append(Y[samp])

        #left shifting
        else:
            #Random number between 1 to 100 ms
            shift = 0.001*np.random.randint(0,100)

            series = X[samp]
            series_shif = np.roll(series,int(-shift*16000))
            X_time_shifted[get]= series_shif
            Y_time_shifted.append(Y[samp])
        
        noise = np.random.randint(X_n.shape[0])
        X_time_shifted[get] = X_time_shifted[get] + X_n[noise]
            
        get+=1

    if samp%100==0:
        print('Done shifting '+str(samp) +'samples')
        
Y_time_shifted = np.array(Y_time_shifted)
Y_time_shifted = Y_time_shifted.reshape(known,1)

In [ ]:
np.save('data/X_known_MORE.npy',X_time_shifted)
np.save('data/Y_known_MORE.npy',Y_time_shifted)

In [ ]:
print(Y_time_shifted[6111])
ipd.Audio(X_time_shifted[6111],rate=16000)